In [1]:
date = '2017-05-02'
iscan_Dspad532 = 20
iscan_Aspad532 = 6
basename = 'two_mantas'

In [2]:
#log_fname = '%s/%s_lv_scan_log2.txt' % (date, basename)
log_fname = '%s_lv_scan_log.txt' % basename
log_fname

'two_mantas_lv_scan_log.txt'

In [3]:
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
import ruamel_yaml

import matplotlib.pyplot as plt
%matplotlib inline

# Read Scan Data

In [4]:
def param_line_to_df(line, scan_params, tidy=False):
    """Convert "Parameters" line of fit results from LV into a DataFrame.
    """
    spot_params_arr = np.array([
            [float(param.replace('[', '')) for param in spot_str.split(',')] 
            for spot_str in line.split(':')[1].strip().split('],')[:-1] ])
    if spot_params_arr.shape[1] == 3:
        spot_params_arr = np.hstack((spotsa, np.zeros((spotsa.shape[0], 2))))
    elif spot_params_arr.shape[1] == 4:
        spot_params_arr = np.hstack((spotsa, np.zeros((spotsa.shape[0], 1))))
    
    columns = ['ampl', 'center', 'sigma', 'offset', 'residue']
    df = (pd.DataFrame(spot_params_arr, columns=columns)
            .assign(residue=lambda x: x.residue)
            .assign(ix=scan_params['coord_x'])
            .assign(iy=scan_params['coord_y']))
    if tidy:
        df = pd.melt(df, id_vars=['ix', 'iy'], var_name='param')
    return df

In [5]:
def load_scan_data(log_fname):
    """
    Read scan text from file and return a dict with keys 'timestamps, values.' 
    """
    with open(log_fname) as f:
        scandata_raw = f.read().strip()
    scandata_list = scandata_raw.split('---\n')[1:]
    print('Number of scans: %d' % len(scandata_list))
    datetime_list = []
    
    datetime_list = []
    for scandata in scandata_list:
        line_list = scandata.split('\n')
        line = line_list[0]
        item_list = line.split(',')
        for item in item_list:
            if 'datetime' in item:
                datetime = item.split(':')[1].strip()            
        datetime_list.append(datetime)   
        
    dict_of_scans = {x: y for x, y in zip(datetime_list, scandata_list)}
    
    return dict_of_scans

In [6]:
def get_scan_params(line):
    """Convert line of scan parameters from LV into a dict.
    """
    scan_params = ruamel_yaml.load('\n'.join(s.strip() for s in line.split(',')))
    scan_params['coord_x'] = np.tile(np.arange(scan_params['nspots_x']), 
                                     scan_params['nspots_y'])
    scan_params['coord_y'] = np.repeat(np.arange(scan_params['nspots_y']), 
                                       scan_params['nspots_x'])
    return scan_params

In [7]:
def process_scan(fit_result_text, ixline=6, iyline=7):
    """
    Read a multi-line scan text from LV and return a DataFrame 
    of fitted parameters and a dict of scan parameters.
    """
    lines = fit_result_text.split('\n')
    scan_params = get_scan_params(lines[0])
    xdf = param_line_to_df(lines[ixline], scan_params).assign(direction='x')
    ydf = param_line_to_df(lines[iyline], scan_params).assign(direction='y')
    df = pd.concat((xdf, ydf))
    df = df.assign(direction=pd.Categorical(df.direction))
    return df, scan_params

# List of all scans

In [8]:
dict_of_scans = load_scan_data(log_fname)
dict_of_scans.keys()

Number of scans: 21


dict_keys(['2017-05-02 12-45-29', '2017-05-02 12-47-11', '2017-05-02 12-47-31', '2017-05-02 12-52-37', '2017-05-02 12-55-44', '2017-05-02 12-56-07', '2017-05-02 12-56-30', '2017-05-02 12-59-45', '2017-05-02 13-01-10', '2017-05-02 13-02-06', '2017-05-02 13-02-28', '2017-05-02 13-02-45', '2017-05-02 13-03-07', '2017-05-02 13-03-20', '2017-05-02 13-04-19', '2017-05-02 15-28-57', '2017-05-02 15-29-51', '2017-05-02 15-30-19', '2017-05-02 15-30-43', '2017-05-02 15-31-01', '2017-05-02 15-31-21'])

In [9]:
for datet, scan in dict_of_scans.items():
    if 'laser: G, ch: G' in scan:
        print(datet)

2017-05-02 12-45-29
2017-05-02 12-47-11
2017-05-02 12-47-31
2017-05-02 15-28-57
2017-05-02 15-29-51
2017-05-02 15-30-19
2017-05-02 15-30-43
2017-05-02 15-31-01
2017-05-02 15-31-21


In [10]:
for datet, scan in dict_of_scans.items():
    if 'laser: G, ch: R' in scan:
        print(datet)

2017-05-02 12-52-37
2017-05-02 12-55-44
2017-05-02 12-56-07
2017-05-02 12-56-30


# Scan Data Analysis

In [11]:
dfD532, scan_params = process_scan(dict_of_scans['2017-05-02 15-31-21'])
scan_params

{'center_x': 2.0,
 'center_y': -67.0,
 'ch': 'G',
 'coord_x': array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]),
 'coord_y': array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]),
 'datetime': '2017-05-02 15-31-21',
 'integration_ms': 200,
 'laser': 'G',
 'nspots_x': 4,
 'nspots_y': 4,
 'pitch_x': 24.3,
 'pitch_y': 23.14,
 'range': 10,
 'step': 0.4,
 'z': 10}

In [12]:
dfD532.head()

,ampl,center,sigma,offset,residue,ix,iy,direction
0,8962.3,-33.35,1.665,11910.8,67288.92,0,0,x
1,12827.8,-9.25,1.630,13671.6,103603.36,1,0,x
2,13552.4,14.92,1.586,14302.2,105898.81,2,0,x
3,12328.2,39.12,1.677,13040.3,126666.61,3,0,x
4,7666.4,-34.00,1.524,13227.2,44112.92,0,1,x


In [13]:
dfA532, scan_params = process_scan(dict_of_scans['2017-05-02 12-56-30'])
scan_params

{'center_x': 2.0,
 'center_y': -67.0,
 'ch': 'R',
 'coord_x': array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]),
 'coord_y': array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]),
 'datetime': '2017-05-02 12-56-30',
 'integration_ms': 200,
 'laser': 'G',
 'nspots_x': 4,
 'nspots_y': 4,
 'pitch_x': 24.3,
 'pitch_y': 23.14,
 'range': 10,
 'step': 0.4,
 'z': 21}

In [14]:
dfA532.head()

,ampl,center,sigma,offset,residue,ix,iy,direction
0,15102.9,-33.50,1.742,19918.9,167553.94,0,0,x
1,20737.1,-9.34,1.677,23114.2,314309.72,1,0,x
2,16334.9,14.74,1.580,16871.5,171775.64,2,0,x
3,22071.3,39.10,1.698,21182.0,338741.92,3,0,x
4,13887.8,-34.08,1.626,22289.2,87368.06,0,1,x


In [15]:
dfD532.to_csv('D_SPAD_532nm.csv')

In [16]:
dfA532.to_csv('A_SPAD_532nm.csv')

In [17]:
pwd

'E:\\Data\\Antonio\\data\\manta\\notebooks\\48-spot_paper\\alignment\\2017-05-02'